In [ ]:
# Notebook parameters
output_dir = ""
requirements_txt = ""

## About the baroclinic wave test

This is a canonical initial value test case for dry dynamical GCM cores (Jablonowski & Williamson, 2006), i.e. the model is run with a perturbation in the midlatitudes of one hemisphere, as an initial condition. No sources or sinks are applied, so the model is expected to crash eventually (in 30 days or so), but the evolution of the wave enables investigation of time-dependent physical behaviour. See references below for expected results and direct comparisons with other models. 

In [ ]:
# Google Colab Setup
if "google.colab" in str(get_ipython()):
    # mount google drive path
    from google.colab import drive 
    drive.mount('/content/drive', force_remount=False)
    # install project requirements.txt in colab env
    if requirements_txt:
        !pip install -r {requirements_txt}

In [ ]:
import os, glob
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

In [ ]:
# if output_dir is not defined, fallback to climate machine settings or cwd
if not output_dir:
    output_dir = os.environ.get("CLIMATEMACHINE_SETTINGS_OUTPUT_DIR", os.getcwd())
    
nc_datasets = glob.glob(os.path.join(output_dir, "**", "BaroclinicWave_AtmosGCMDefault*.nc"), recursive=True)
if not nc_datasets:
    raise Exception("No NetCDF dataset found under path prefix: `{}`".format(output_dir))
print(nc_datasets)

nc_datasets_s = glob.glob(os.path.join(output_dir, "**", "BaroclinicWave_AtmosGCMSpectra*.nc"), recursive=True)
if not nc_datasets_s:
    raise Exception("No NetCDF dataset found under path prefix: `{}`".format(output_dir))
print(nc_datasets_s)


In [ ]:
data = nc.Dataset(nc_datasets[0],"r")
data_s = nc.Dataset(nc_datasets_s[0],"r")

data

In [ ]:
lon = data["long"][:].data
lat = data["lat"][:].data
alt = data["level"][:].data/1e3
time = data["time"][:].data

alt_index_lo = np.where(alt==1.0)[0][0]
print(alt[alt_index_lo])
alt_index_hi = np.where(alt==10.0)[0][0]
print(alt[alt_index_hi])
alt_index_s = np.where(alt==4.0)[0][0]
print(alt[alt_index_s])

## Plots 
- upper-level lon/lat plots: u, v, vort
- surface lon/lat plots: T, p 
- upper-level EKE power spectrum 


In [ ]:
fig_, ax_ = plt.subplots(6,3, figsize=(24,12))
for ii,it in enumerate([6, 8, 10]):
    u = data["u"][it,alt_index_hi,:,:].data
    v = data["v"][it,alt_index_hi,:,:].data
    T = data["temp"][it,alt_index_lo,:,:].data
    p = data["pres"][it,alt_index_lo,:,:].data/1e2
    p = p - p.mean(axis=1, keepdims=True)
    vort = data["vort"][it,alt_index_hi,:,:]*1e5
    spec = data_s["spectrum_2d"][it,alt_index_s,:,:].data
    
    C = ax_[0,ii].contourf(lon, lat, u, cmap = 'RdBu_r')
    fig_.colorbar(C, ax=ax_[0,ii])
    ax_[0,ii].set_title("u @%skm: Day %s"%(alt[alt_index_hi],it))
    
    C = ax_[1,ii].contourf(lon, lat, v, cmap = 'RdBu_r')
    fig_.colorbar(C, ax=ax_[1,ii])
    ax_[1,ii].set_title("v @%skm"%alt[alt_index_hi])
    
    C = ax_[2,ii].contourf(lon, lat, T, cmap = 'RdBu_r')
    fig_.colorbar(C, ax=ax_[2,ii])
    ax_[2,ii].set_title("temperature @%skm"%alt[alt_index_lo])
    
    C = ax_[3,ii].contourf(lon, lat, p, cmap = 'RdBu_r')
    fig_.colorbar(C, ax=ax_[3,ii])
    ax_[3,ii].set_title("pressure ano @%skm"%alt[alt_index_lo])

    C = ax_[4,ii].contourf(lon, lat, vort, cmap = 'RdBu_r')
    fig_.colorbar(C, ax=ax_[4,ii], )
    ax_[4,ii].set_title("vorticity*1e5 @%skm"%alt[alt_index_hi])

    C = ax_[5,ii].plot(np.arange(1,np.shape(spec)[0]+1), np.sum(spec,axis=1))
    ax_[5,ii].set_title("EKE power spectrum @%skm"%alt[alt_index_hi])
    plt.yscale("log")
    plt.xscale("log")
        
    for jj in np.arange(5):
        ax_[jj,ii].set_ylim([0, 90])
        if jj<5:
            ax_[jj,ii].set_xticklabels([])
    
#plt.show()
plt.savefig('./BW_plot.png', facecolor='w', transparent=False)

## References:
- Jablonowski and Williamson (2006) https://pdfs.semanticscholar.org/cf27/9eab20507e284f4a22dd2134458adc2f0c08.pdf
- Jablonowski and Williamson (2011) https://link.springer.com/chapter/10.1007/978-3-642-11640-7_13
- Ullrich et al. (2013) https://rmets.onlinelibrary.wiley.com/doi/full/10.1002/qj.2241 